In [2]:
import numpy as np
import matplotlib.pyplot as plt
import copy


In [3]:
## Helper functions ##

def printnp(matrix):
    ## Print matrix with 12 decimals
    print(np.array2string(matrix, formatter={'float_kind': '{0:.12f}'.format}))

def read_enuc(path):
    
    enuc = 0
    with open(path, 'r') as f:
        for line in f:
            enuc = float(line)
    return enuc

def read_one_e_integral(path, nAOs, description):

    # open and append to a numpy list
    one_e_mat = np.zeros([nAOs, nAOs])
    infile = open(path, 'r')

    for line in infile:
        idx1 , idx2, val = line.split()
        i, j = int(idx1) - 1, int(idx2) - 1
        val = float(val)
        one_e_mat[i,j] = val
        one_e_mat[j,i] = val
    infile.close()    
    print(f"Read the {description} matrix successfully!")
    return one_e_mat

def read_two_e_integral(path, nAOs, description):

    two_e_mat = np.zeros([nAOs, nAOs, nAOs, nAOs])
    infile = open(path, 'r')

    for line in infile:
        idx1, idx2, idx3, idx4, val = line.split()
        i,j,k,l = int(idx1) - 1, int(idx2) - 1, int(idx3) - 1, int(idx4) - 1
        val = float(val)

        two_e_mat[i,j,k,l] = val
        two_e_mat[j,i,k,l] = val
        two_e_mat[i,j,l,k] = val
        two_e_mat[j,i,l,k] = val

        two_e_mat[k,l,i,j] = val
        two_e_mat[l,k,i,j] = val
        two_e_mat[k,l,j,i] = val
        two_e_mat[l,k,j,i] = val
    infile.close()
    print(f"Read the {description} matrix successfully!")
    return two_e_mat

In [39]:
def canonical_hartree_fock(h_core, eri_tensor, overlap_matrix, enuc, nelectrons):

    def get_F(density_matrix, core_ham, eri_tensor):
        fock = core_ham.copy()
        fock = fock - 1/2 * np.einsum("mnls,nl->ms", eri_tensor, density_matrix, optimize = True)
        fock = fock + np.einsum("mnls,ls->mn", eri_tensor, density_matrix, optimize = True)
        return fock
    def get_X(overlap_matrix):

        eivals, eivecs = np.linalg.eigh(overlap_matrix)
        trans = eivals**(-1/2)
        return eivecs @ np.diag(trans) @ np.transpose(np.conjugate(eivecs))

    nAOs = h_core.shape[0]
    X = get_X(overlap_matrix)
    #print(X @ overlap_matrix @ np.transpose(X))
    focc = np.zeros([nAOs,nAOs])
    focc[:nelectrons,:nelectrons] = np.identity(nelectrons)
    # Initial guess of the coefficient matrix
    F_trans = np.conjugate(np.transpose(X)) @ h_core @ X
    eivals, eivecs = np.linalg.eigh(F_trans)
    C = X @ eivecs
    P = 2 * C @ focc @ np.transpose(np.conjugate(C))
    F = get_F(P, h_core, eri_tensor)
    Ei, Ef = 0, 0.5 * np.trace(P @ (h_core + F))  
    # Fixed point iteration (This scheme ensures that the final C is canonical orbitals that diagonalize the Fock matrix)
    while abs(Ei - Ef) > 1e-9:
        Ei = Ef
        print("==================================")
        print(f"The current energy is {Ei + enuc}")
        eivals, eivecs = np.linalg.eigh(np.conjugate(np.transpose(X)) @ F @ X)
        C = X @ eivecs 
        P = 2 * C @ focc @ np.transpose(np.conjugate(C))
        F = get_F(P, h_core, eri_tensor)
        Ef = 0.5 * np.trace(P @ (h_core + F))

    return eivals, C, Ef + enuc

In [41]:
# I think we will parallelize this part of the code

def eri_tensor_to_MO(eri_tensor_AO, coeff_mat):
    # Sequential contraction
    eri1 = np.einsum("mnls,mp->pnls",eri_tensor_AO, coeff_mat, optimize=True)
    eri2 = np.einsum("pnls,nq->pqls",eri1, coeff_mat, optimize=True)
    eri3 = np.einsum("pqls,lr->pqrs",eri2, coeff_mat, optimize=True)
    eri_tensor_MO = np.einsum("pqrs,st->pqrt",eri3, coeff_mat, optimize=True)
    return eri_tensor_MO

def mp2_correction(eri_tensor_MO, orbital_energy, nelectrons):
    
    nAOs = orbital_energy.shape[0]
    mp2_correlation = 0
    for i in range(nelectrons):
        for j in range(nelectrons):
            for a in range(nelectrons, nAOs):
                for b in range(nelectrons, nAOs):
                    mp2_correlation += eri_tensor_MO[i,a,j,b] * ( 2 * eri_tensor_MO[i,a,j,b] - eri_tensor_MO[i,b,j,a]) / (orbital_energy[i] + orbital_energy[j] - orbital_energy[a] - orbital_energy[b])

    return mp2_correlation


In [43]:
t_path = "test/h2o/STO-3G/t.dat"
v_path = "test/h2o/STO-3G/v.dat"
eri_path = "test/h2o/STO-3G/eri.dat"
overlap_path = "test/h2o/STO-3G/s.dat" 
enuc_path = "test/h2o/STO-3G/enuc.dat"

nbasis, nelectrons = 7, 5

# The integrals can be calculated from another software

t_matrix = read_one_e_integral(t_path, nbasis, "kinetic")
v_matrix = read_one_e_integral(v_path, nbasis, "nuclear attraction")
overlap_matrix = read_one_e_integral(overlap_path, nbasis, "overlap (S)")
enuc = read_enuc(enuc_path)
eri_tensor = read_two_e_integral(eri_path, nbasis, "ERI")


# Energy of water in STO-3G basis: -74.942079928192, in DZP basis: -76.008821792900, in DZ absis: -75.977878975377

orbital_energy, opt_orbitals, HF_Energy = canonical_hartree_fock(t_matrix + v_matrix, eri_tensor, overlap_matrix, enuc, nelectrons)

eri_tensor_MO = eri_tensor_to_MO(eri_tensor, opt_orbitals)
mp2 = mp2_correction(eri_tensor_MO, orbital_energy, nelectrons)
print(f"mp2 correction to the hartree fock energy is {mp2}")

Read the kinetic matrix successfully!
Read the nuclear attraction matrix successfully!
Read the overlap (S) matrix successfully!
Read the ERI matrix successfully!
The current energy is -73.28579642110032
The current energy is -74.82812537974482
The current energy is -74.93548799801144
The current energy is -74.94147774249389
The current energy is -74.94197199611565
The current energy is -74.94205606141364
The current energy is -74.94207441913818
The current energy is -74.94207864763922
The current energy is -74.94207963013933
The current energy is -74.94207985880342
The current energy is -74.94207991203719
The current energy is -74.94207992443071
The current energy is -74.94207992731614
mp2 correction to the hartree fock energy is -0.04914998959623713
